In [2]:
import pandas as pd

In [3]:
books = pd.read_json("../local/raw_data/books.jsn")
books["serial_name"] = books["serial_name"].apply(lambda x: x.lower() if x else "")
books["material_name"] = books["serial_name"].apply(lambda x: x.lower() if x else "")
books["rubric_name"] = books["serial_name"].apply(lambda x: x.lower() if x else "")
books["main"] = (books["author_fullName"].str.lower() +
                 " [SEP] " +
                 books["title"].apply(
                     lambda x: x.lower().split(":")[0].strip()
                 ))
# group by author + book name
books = (books.groupby(["main"])
    .agg(
        bbk=("bbk", "first"),
        title=("title", "first"),
        serial_name=("serial_name", "max"),
        material_name=("material_name", "max"),
        annotation=("annotation", "max"),
        rubric_name=("rubric_name", "max"),
        author_fullName=("author_fullName", "first"),
        id=(
            "id",
            lambda x: sorted(list(set(x))),
        ),
    )
    .reset_index()
)

books["text"] = (books["main"] + " [SEP] " +
                 books["rubric_name"].str.lower() + " [SEP] " +
                 books["serial_name"].str.lower() + " [SEP] " +
                 books["material_name"].str.lower() + " [SEP] " +
                 books["annotation"].str.lower())
books.to_csv("text_csv.csv", index=None)
books_dict = dict()
for i, row in books.iterrows():
    ids = row["id"]
    for book_id in ids:
        books_dict[book_id] = row["text"]
books

,main,bbk,title,serial_name,material_name,annotation,rubric_name,author_fullName,id,text
0,"clark mary higgins [SEP] weep no more, my lady",84,"Weep No More, My Lady",,,,,Clark Mary Higgins,[1177634],"clark mary higgins [SEP] weep no more, my lady..."
1,connelly michael [SEP] a darkness more than night,81.2Англ,A Darkness More Than Night : a novel; [кн. на ...,,,A Darkness More Than Night is the tenth novel ...,,Connelly Michael,[1627543],connelly michael [SEP] a darkness more than ni...
2,connelly michael [SEP] city of bones,81.2Англ,City of Bones : [кн. на англ. яз.],,,"On New Year's Day, Detective Harry Bosch field...",,Connelly Michael,[1627533],connelly michael [SEP] city of bones [SEP] [S...
3,connelly michael [SEP] echo park,84,Echo Park,,,,,Connelly Michael,[1178414],connelly michael [SEP] echo park [SEP] [SEP] ...
4,connelly michael [SEP] the drop,84,The Drop,,,,,Connelly Michael,[1028952],connelly michael [SEP] the drop [SEP] [SEP] ...
...,...,...,...,...,...,...,...,...,...,...
3833,ярышевская елена николаевна [SEP] вышел пудель...,Д,Вышел пудель погулять,строчка за строчкой с сыном и дочкой,строчка за строчкой с сыном и дочкой,"Вы знаете, какие собаки считаются самыми мален...",строчка за строчкой с сыном и дочкой,Ярышевская Елена Николаевна,[1951187],ярышевская елена николаевна [SEP] вышел пудель...
3834,ясницкий леонид нахимович [SEP] современные пр...,72,Современные проблемы науки : учебное пособие,,,Рассмотрены вопросы становления и эволюции нау...,,Ясницкий Леонид Нахимович,[1758200],ясницкий леонид нахимович [SEP] современные пр...
3835,яхина гузель шамилевна [SEP] дети мои,84(2Рос)6,Дети мои : роман,проза гузель яхиной,проза гузель яхиной,"Новый роман лауреата премий ""Большая книга"" и ...",проза гузель яхиной,Яхина Гузель Шамилевна,"[324421, 1512269]",яхина гузель шамилевна [SEP] дети мои [SEP] пр...
3836,яхина гузель шамилевна [SEP] зулейха открывает...,84(2Рос)6,Зулейха открывает глаза,проза гузель яхиной,проза гузель яхиной,"Гузель Яхина родилась и выросла в Казани, окон...",проза гузель яхиной,Яхина Гузель Шамилевна,"[1389555, 1814545]",яхина гузель шамилевна [SEP] зулейха открывает...


In [20]:
!iconv -f cp1251 < drive/MyDrive/libraries/datasets_biblioteki/datasets_2/siglas.csv | head

id;cbs;adress;name;eisk;;
1;3;ул. Клары Цеткин, д. 11, корп. 1;21-1;284375696001;;
2;3;ул. Космонавта Волкова, д. 3;21-2;284375229001;;
3;3;Ленинградское ш., д. 114;22-1;284384346001;;
4;3;Ленинградское ш., д. 120;22-2;284384353001;;
5;3;ул. Астрадамская, д. 4;23-1;284385857001;;
6;3;ул. Тимирязевская, д. 17, корп. 1;23-2;284385867001;;
7;3;ул. Новопесчаная, д. 23 корп.7;24;284385847001;;
8;3;ул. Врубеля, д. 13;25;284385826001;;
9;3;ул. Зеленоградская, д. 27А;26;284385821001;;


In [21]:
!iconv -f cp1251 < drive/MyDrive/libraries/datasets_biblioteki/datasets_2/cat_1.csv | head

recId;aut;title;place;publ;yea;lan;rubrics;person;serial;material;biblevel;ager
1;Устинова Татьяна Витальевна;Призрак Канта;Москва;Эксмо;2018;rus;Художественная литература : Российский детектив;;Татьяна Устинова. Первая среди лучших;Текст. материал;Монография;16+
2;;Наука и жизнь;Москва;Редакция журнала «Наука и жизнь»;1938-;rus;Наука. Науковедение;;;Текст. материал;Периодика;12+
3;;Наука и жизнь;;;2018;rus;Наука. Науковедение;;;Текст. материал;Монография;12+
4;;Наука и жизнь;;;2018;rus;Наука. Науковедение;;;Текст. материал;Монография;12+
5;;Наука и жизнь;;;2018;rus;Наука. Науковедение;;;Текст. материал;Монография;12+
6;Тараканов Борис Игоревич;Кольцо времени;Москва;АиФ Принт;2001;rus;Художественная литература : Фэнтэзи;;Черная звезда;Текст. материал;Монография;
8;;Фантастика-86;Москва;Молодая гвардия;1986;;Художественная литература;;;Текст. материал;Монография;
9;;Фантастика-90;Москва;Молодая гвардия;1990;;Художественная литература : Фантастика;;;Текст. материал;Монография;
10;Федотов

In [10]:
interactions = pd.read_excel("../local/raw_data/dataset_knigi_1.xlsx")
interactions["book_id"] = interactions["source_url"].str.split("/").str[-2].astype(int)
interactions["book_str"] = interactions["book_id"].replace(books_dict)
print(interactions["event"].unique())
interactions

['add' 'create_order']


,source_url,event,dt,user_id,book_id,book_str
0,https://www.mos.ru/knigi/book/1163786/,add,2021-06-01,1,1163786,бэйли гвен [SEP] о чем думает ваша собака? [SE...
1,https://www.mos.ru/knigi/book/66612/,add,2021-06-01,1,66612,яковлев юрий яковлевич [SEP] у человека должна...
2,https://www.mos.ru/knigi/book/1034596/,add,2021-06-01,1,1034596,"баранова марта петровна [SEP] тяпа, борька и р..."
3,https://www.mos.ru/knigi/book/1152760/,add,2021-06-01,1,1152760,"дункан дэвид дуглас [SEP] лумп - собака, котор..."
4,https://www.mos.ru/knigi/book/16168/,add,2021-06-01,1,16168,брюгген мария ван [SEP] почему собака лает и е...
...,...,...,...,...,...,...
4181,https://www.mos.ru/knigi/book/1024959/,create_order,2021-03-30,103,1024959,носов николай [SEP] незнайка на луне [SEP] [S...
4182,https://www.mos.ru/knigi/book/915148/,create_order,2021-03-30,103,915148,поликарпова татьяна николаевна [SEP] листья бу...
4183,https://www.mos.ru/knigi/book/1859007/,create_order,2021-03-30,103,1859007,лабковский михаил александрович [SEP] про любо...
4184,https://www.mos.ru/knigi/book/218302/,create_order,2021-03-31,103,218302,"андерсон пол уильям [SEP] операция ""хаос""; три..."


In [11]:
# interactions[interactions["event"] == "create_order"]
mos_user_book_dict = (
    interactions.groupby("user_id")
                .apply(lambda x: set(
                                 list(x["book_id"])
                                 )
                )
                .to_dict()
                )
str(list(mos_user_book_dict.items())[1])

'(2, {1224064, 1652993, 1648003, 1323524, 2097031, 1863048, 12556, 1653518, 42235, 2496376, 2522648, 735129, 1600025, 2496123, 2088604, 2113053, 1653149, 1429919, 2024605, 1445920, 2111270, 1588519, 1801641, 1650986, 1652011, 1055787, 1652350, 2005551, 266291, 1569973, 871094, 2111674, 1595834, 352956, 265277, 1594174, 274367, 1863356, 265283, 2111686, 1554631, 2510920, 1554632, 1610570, 871755, 1554637, 1554638, 2109774, 2107090, 1863382, 1954135, 1238, 2005595, 961371, 1565662, 267615, 1853920, 2522594, 1210981, 1001703, 2522473, 12651, 2119021, 1957742, 1962235, 6128, 1847411, 1818996, 906231, 6135, 2111224, 1721723, 1421694})'

In [13]:
circulaton_df = pd.read_csv('../local/raw_data/circulations.csv')
circulaton_df.head()

,user_id,book_id,dt
0,118736,5186,01.09.2019
1,118736,316682,01.09.2019
2,180893,751608,01.09.2019
3,201318,585605,01.09.2019
4,201318,41656,01.09.2019


In [14]:
circulation_reader_book_dict = (
    circulaton_df.groupby("user_id")
                 .apply(lambda x: set(list(x["book_id"])))
                 .to_dict()
)

In [16]:
# матчи пользователей из циркулатона
# с пользователями мос.ру по айдишникам книжек
mos_circ_user = dict()
for user, user_books in mos_user_book_dict.items():
    max_intersect = 0
    max_circ_user = None
    circ_books_total = 0
    for circ_reader, circ_books in circulation_reader_book_dict.items():
        intersection = len(user_books.intersection(circ_books))
        if intersection > max_intersect:
            max_intersect = intersection
            max_circ_user = circ_reader
            circ_books_total = len(circ_books)
    if max_intersect > 0:
        mos_circ_user[user] = {"circ_user": max_circ_user,
                            "intersection": max_intersect,
                            "circ_books_total": circ_books_total,
                            "mos_perc": max_intersect / len(user_books),
                            "circ_perc": max_intersect / circ_books_total,
                            "mos_books": len(user_books)
                            }
        print(f"\r{user}: {str(mos_circ_user[user])}", end="")
mos_circ_user_df = pd.DataFrame(mos_circ_user).T
mos_circ_user_df

103: {'circ_user': 539903, 'intersection': 2, 'circ_books_total': 7, 'mos_perc': 0.03076923076923077, 'circ_perc': 0.2857142857142857, 'mos_books': 65}}3}

,circ_user,intersection,circ_books_total,mos_perc,circ_perc,mos_books
1,647497.0,9.0,11.0,0.067669,0.818182,133.0
2,135729.0,4.0,8.0,0.054795,0.500000,73.0
3,6336.0,1.0,7.0,0.071429,0.142857,14.0
4,14613.0,1.0,7.0,0.076923,0.142857,13.0
5,18394.0,2.0,6.0,0.023810,0.333333,84.0
...,...,...,...,...,...,...
99,39853.0,1.0,4.0,0.090909,0.250000,11.0
100,425231.0,1.0,5.0,0.052632,0.200000,19.0
101,219058.0,5.0,12.0,0.131579,0.416667,38.0
102,16352.0,2.0,10.0,0.090909,0.200000,22.0


In [18]:
mos_circ_user_df[mos_circ_user_df['circ_perc'] == 1]

,circ_user,intersection,circ_books_total,mos_perc,circ_perc,mos_books
29,6434.0,1.0,1.0,0.090909,1.0,11.0
61,4209.0,2.0,2.0,0.068966,1.0,29.0


In [19]:
mos_circ_user_df[mos_circ_user_df['mos_perc'] == 1]

,circ_user,intersection,circ_books_total,mos_perc,circ_perc,mos_books
98,648840.0,5.0,9.0,1.0,0.555556,5.0
